See annotated [K20P64M72SF1RM][1] manual (MK20DX128 processor used on [Teensy 3.1/3.2][2]).

[1]: https://xodo.com/app/d/?/=9efbee54-ee85-4772-8e5c-859fcb213d74
[2]: https://www.pjrc.com/teensy/

--------------------------------------------------

In [142]:
import pandas as pd

# ADC registers memory map on page 651 in `K20P64M72SF1RM.pdf`.
ADC_START_ADDRESS = pd.Series([0x4003B000, 0x400BB000], index=[0, 1])

ADC_REGISTER_MAP = (pd.DataFrame([['SC1A', 0x00000000, 0x0000001F],
                                  ['SC1B', 0x00000004, 0x0000001F],
                                  ['CFG1', 0x00000008, 0x00000000],
                                  ['CFG2', 0x0000000C, 0x00000000],
                                  ['RA'  , 0x00000010, 0x00000000],
                                  ['RB'  , 0x00000014, 0x00000000],
                                  ['CV1' , 0x00000018, 0x00000000],
                                  ['CV2' , 0x0000001C, 0x00000000],
                                  ['SC2' , 0x00000020, 0x00000000],
                                  ['SC3' , 0x00000024, 0x00000000],
                                  ['OFS' , 0x00000028, 0x00000004],
                                  ['PG'  , 0x0000002C, 0x00008200],
                                  ['MG'  , 0x00000030, 0x00008200],
                                  ['CLPD', 0x00000034, 0x0000000A],
                                  ['CLPS', 0x00000038, 0x00000020],
                                  ['CLP4', 0x0000003C, 0x00000200],
                                  ['CLP3', 0x00000040, 0x00000100],
                                  ['CLP2', 0x00000044, 0x00000080],
                                  ['CLP1', 0x00000048, 0x00000040],
                                  ['CLP0', 0x0000004C, 0x00000020],
                                  ['PGA' , 0x00000050, 0x00000000],
                                  ['CLMD', 0x00000054, 0x0000000A],
                                  ['CLMS', 0x00000058, 0x00000020],
                                  ['CLM4', 0x0000005C, 0x00000200],
                                  ['CLM3', 0x00000060, 0x00000100],
                                  ['CLM2', 0x00000064, 0x00000080],
                                  ['CLM1', 0x00000068, 0x00000040],
                                  ['CLM0', 0x0000006C, 0x00000020]],
                                  columns=['register_name', 'address_offset',
                                           'reset_value'])
                    .set_index('register_name'))


def default_adc_reg(adc_num):
    ADC_REG = ADC_REGISTER_MAP.copy()
    ADC_REG.address_offset += ADC_START_ADDRESS[adc_num]
    ADC_REG.rename(columns={'address_offset': 'address', 'reset_value': 'value'}, inplace=True)
    return ADC_REG


ADC0_REG = default_adc_reg(0)
ADC1_REG = default_adc_reg(1)
hex(ADC_START_ADDRESS[1] - ADC_START_ADDRESS[0])

'0x80000L'

# ADC_SC1m: ADC Status and Control Registers 1

Page 653.

 - `COCO`: Conversion complete flag
 - `AIEN`: Interrupt enable
 - `DIFF`: Differential mode enable
 - `ADCH`: 4 bits, Input channel select
 
 
## Teensy 3.1/3.2 (MK20DX256)

The MK20DX256 processor has two ADC status and control registers.
Each register is associated with a pre-trigger in the Programmable Delay Block.

# ADC_CFG1: ADC Configuration Register 1

Page 656

 - `ADLPC`: Low-power configuration
 - `ADIV`: 2 bits, Clock divide select
     * 0: /1, 1: /2, 2: /4, 3: /8
 - `ADLSMP`: Sample time configuration
     * 0: Short sample time, 1: Long sample time
 - `MODE`: 2 bits, Conversion mode selection
     * 0: 8-bit, 1: 12-bit, 2: 10-bit, 3: 16-bit
 - `ADICLK`: 2 bits, Input clock select
     * 0: Bus clock, 1: Bus clock/2, 2: Alternate clock (ALTCLK), 3: Asynchronous clock (ADACK)

# ADC_CFG2: ADC Configuration Register 2

Page 658.

 - `MUXSEL`: ADC mux select
     * 0: ADxxa channels selected, 1: ADxxb channels selected
 - `ADACKEN`: Asynchronous clock output enable
 - `ADHSC`: High-speed configuration
 - `ADLSTS`: Long sample time select
     * 0: 20 extra ADCK cycles (default)
     * 1: 12 extra ADCK cycles
     * 2: 6 extra ADCK cycles
     * 3: 2 extra ADCK cycles

# ADC_SC2: Status and Control Register 2

Page 661.

 - `ADACT`: Conversion active
 - `ADTRG`: Conversion trigger select
     * 0: Software trigger, 1: Hardware trigger
 - `ACFE`: Compare Function Enable
 - `ACFGT`: Compare function greater than enable
 - `ACREN`: Compare function range enable
 - `DMAEN`: DMA enable
     * 1: 1 DMA is enabled and will assert the ADC DMA request during an ADC conversion complete
      event noted when any of the `SC1n[COCO]` flags is asserted.
 - `REFSEL`: 2 bits, Voltage reference selection
     * 0: Default, 1: Alternate, 2-3: Reserved
     
     
## Teensy ADC_Module ##

The `startPDB` method sets `ADTRG=1` to enable trigger of ADC using Programmable Delay Block.
See "Programmable Delay Block notes" notebook for details.

Voltage reference selection; 0: Default, 1: Alternate, 2-3: Reserved

# ADC_SC3: Status and Control Register 3

Page 663.

 - `CAL`: Calibration
 - `CALF`: Calibration failed flag
 - `ADCO`: Continuous conversion enable
 - `AVGE`: Hardware average enable
 - `AVGS`: Hardware average select
     * Samples averaged - 0: 4, 1: 8, 0: 16, 0: 32

# ADC_PGA: ADC PGA Register

Page 670.

 - `PGAEN`: PGA enable
 - `PGALPb`: PGA low-power mode control
 - `PGAG`: 19-16, PGA Gain setting, $PGA~gain = 2^{PGAG}$
     * 0: 1, 1: 2, 2: 4, 3: 8, 4: 16, 5: 32, 6: 64